In [1]:
import pandas as pd  # Importa la librería pandas y la asigna al alias 'pd', utilizada para la manipulación y análisis de datos.
import time  # Importa el módulo time, que proporciona diversas funciones relacionadas con la manipulación del tiempo, como delays y medición de intervalos.
import json
import numpy as np
import random
import time
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from pandas import json_normalize

from selenium import webdriver  # Importa el módulo webdriver de Selenium, utilizado para automatizar navegadores web.
from selenium.webdriver.common.by import By  # Importa el módulo By, que proporciona métodos para localizar elementos en una página web por diferentes atributos (ID, nombre, clase, etc.).
from selenium.webdriver.support import expected_conditions as EC  # Importa el módulo expected_conditions, que contiene una serie de condiciones que se pueden esperar (esperar a que un elemento sea clicable, visible, etc.).
from selenium.webdriver.support.ui import WebDriverWait  # Importa el módulo WebDriverWait, que permite esperar hasta que se cumpla una condición específica.
from selenium.webdriver.chrome.service import Service  # Importa el módulo Service, que permite iniciar y controlar el servicio del navegador Chrome.
from selenium.webdriver.chrome.options import Options  # Importa el módulo Options, que permite personalizar las opciones de configuración del navegador Chrome.
from webdriver_manager.chrome import ChromeDriverManager  # Importa el ChromeDriverManager, que gestiona automáticamente la descarga e instalación del controlador de Chrome.
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
# Configuración
import warnings  # Importa el módulo warnings, que maneja los mensajes de advertencia en Python.
warnings.filterwarnings("ignore")  # Configura para ignorar todas las advertencias.
warnings.simplefilter(action='ignore', category=FutureWarning)  # Configura para ignorar las advertencias de tipo FutureWarning, que se utilizan para notificar sobre cambios que pueden ocurrir en versiones futuras de las librerías.
pd.set_option('display.max_columns', None)  # Configura pandas para mostrar todas las columnas de un DataFrame al imprimirlo, sin truncar la visualización.
pd.set_option('display.max_rows', None)  # Configura pandas para mostrar todas las filas de un DataFrame al imprimirlo, sin truncar la visualización.

In [4]:
def random_sleep_time():
    return random.uniform(3, 6)

def setup_driver():
    driver = webdriver.Chrome()
    return driver

In [5]:

def close_ad_popup(driver):
    try:
        # Esperar hasta 10 segundos para que aparezca la publicidad
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "webpush-swal2-container"))
        )
        
        # Intentar cerrar con el botón 'x'
        try:
            close_button = driver.find_element(By.CLASS_NAME, "webpush-swal2-close")
            close_button.click()
            print("Publicidad cerrada exitosamente con el botón 'x'.")
            return
        except NoSuchElementException:
            print("No se encontró el botón 'x' para cerrar la publicidad.")
        
        # Si no se pudo cerrar con 'x', intentar con el botón "FREE TRIAL"
        try:
            free_trial_button = driver.find_element(By.XPATH, "//button[contains(@class, 'webpush-swal2-confirm')]")
            free_trial_button.click()
            print("Publicidad cerrada exitosamente con el botón 'FREE TRIAL'.")
            return
        except NoSuchElementException:
            print("No se encontró el botón 'FREE TRIAL'.")
        
        # Si aún no se ha cerrado, intentar hacer clic fuera de la publicidad
        try:
            overlay = driver.find_element(By.CLASS_NAME, "webpush-swal2-container")
            driver.execute_script("arguments[0].click();", overlay)
            print("Se intentó cerrar la publicidad haciendo clic fuera de ella.")
        except Exception as e:
            print(f"No se pudo cerrar la publicidad haciendo clic fuera: {str(e)}")
        
    except TimeoutException:
        print("No se encontró la publicidad o no apareció en el tiempo esperado.")
    except Exception as e:
        print(f"Error inesperado al intentar cerrar la publicidad: {str(e)}")

In [6]:
def extract_matches_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    matches = []
    
    date_accordions = soup.find_all('div', class_='Accordion-module_accordion__UuHD0')
    
    for accordion in date_accordions:
        date = accordion.find('div', class_='Accordion-module_header__HqzWD').text.strip()
        match_divs = accordion.find_all('div', class_='Match-module_match__XlKTY')
        
        for match in match_divs:
            try:
                time_or_status = match.find('span', class_=['Match-module_startTime__c49c8', 'Match-module_FT__2rmH7'])
                time_or_status = time_or_status.text.strip() if time_or_status else "N/A"
                
                teams = match.find_all('a', class_='Match-module_teamNameText__Dqv-G')
                home_team = teams[0].text.strip() if teams else "N/A"
                away_team = teams[1].text.strip() if len(teams) > 1 else "N/A"
                
                score_element = match.find('a', class_='Match-module_score__5Ghhj')
                match_id = score_element['id'].split('-')[1] if score_element else "N/A"
                
                scores = score_element.find_all('span') if score_element else []
                home_score = scores[0].text.strip() if scores else "N/A"
                away_score = scores[1].text.strip() if len(scores) > 1 else "N/A"
                
                odds = match.find_all('span', class_='OddsButton-module_oddsText__WD5Dv')
                home_odds = odds[0].text.strip() if odds else "N/A"
                draw_odds = odds[1].text.strip() if len(odds) > 1 else "N/A"
                away_odds = odds[2].text.strip() if len(odds) > 2 else "N/A"
                
                matches.append({
                    'date': date,
                    'time_or_status': time_or_status,
                    'home_team': home_team,
                    'away_team': away_team,
                    'match_id': match_id,
                    'home_score': home_score,
                    'away_score': away_score,
                    'home_odds': home_odds,
                    'draw_odds': draw_odds,
                    'away_odds': away_odds
                })
            except Exception as e:
                print(f"Error procesando un partido: {str(e)}")
    
    return pd.DataFrame(matches)


In [9]:

def scrape_fixtures():
    driver = setup_driver()
    fixtures_url = "https://www.whoscored.com/Regions/250/Tournaments/12/Seasons/10456/Stages/23663/Fixtures/Europe-Champions-League-2024-2025"
    driver.get(fixtures_url)

    all_matches_df = pd.DataFrame()

    try:
        print("Esperando que la página cargue completamente...")
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        
        print("Intentando cerrar la publicidad...")
        # close_ad_popup(driver)
        
        print("Intentando hacer clic en el botón del calendario...")
        calendar_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "toggleCalendar"))
        )
        calendar_button.click()
        
        print("Esperando que el calendario se abra...")
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "datePicker")))
        
        print("Calendario abierto exitosamente.")
        
        month_index = 1
        while True:
            try:
                print(f"Intentando procesar el mes {month_index}")
                
                # Intentar localizar y hacer clic en el mes
                month_selector = f".datePicker_selectable:nth-child({month_index})"
                month = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, month_selector))
                )
                month.click()
                time.sleep(5)  # Esperar a que la página se actualice

                print(f"Extrayendo datos para el mes {month_index}")
                page_source = driver.page_source
                matches_df = extract_matches_from_html(page_source)
                all_matches_df = pd.concat([all_matches_df, matches_df], ignore_index=True)
                print(f"Datos extraídos para el mes {month_index}. Partidos en este mes: {len(matches_df)}")

                # Volver a abrir el calendario para la siguiente iteración
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "toggleCalendar"))).click()
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "datePicker")))
                
                month_index += 1
            
            except TimeoutException:
                print(f"No se encontró el mes {month_index}. Finalizando la extracción.")
                break
            except StaleElementReferenceException:
                print(f"Elemento obsoleto encontrado para el mes {month_index}. Reintentando...")
                continue
            except Exception as e:
                print(f"Error inesperado procesando el mes {month_index}: {str(e)}")
                print("Intentando continuar con el siguiente mes...")
                month_index += 1
                if month_index > 12:  # Asumiendo que no habrá más de 12 meses seleccionables
                    break

    except Exception as e:
        print(f"Se produjo un error general: {str(e)}")
    finally:
        driver.quit()

    return all_matches_df

In [ ]:
# Ejecución principal
all_fixtures_df = scrape_fixtures()
print(f"Total de partidos extraídos: {len(all_fixtures_df)}")
all_fixtures_df.head()

In [12]:
driver = setup_driver()
fixtures_url = "https://www.whoscored.com/Regions/250/Tournaments/12/Seasons/10456/Stages/23663/Fixtures/Europe-Champions-League-2024-2025"
driver.get(fixtures_url)

In [13]:
all_matches_df = pd.DataFrame()

print("Esperando que la página cargue completamente...")
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

print("Intentando cerrar la publicidad...")

Esperando que la página cargue completamente...
Intentando cerrar la publicidad...


In [14]:
print("Intentando hacer clic en el botón del calendario...")
calendar_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "toggleCalendar"))
)
calendar_button.click()

Intentando hacer clic en el botón del calendario...


In [15]:
print("Esperando que el calendario se abra...")
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "datePicker")))

print("Calendario abierto exitosamente.")

month_index = 1
while True:
    try:
        print(f"Intentando procesar el mes {month_index}")
        
        # Intentar localizar y hacer clic en el mes
        month_selector = f".datePicker_selectable:nth-child({month_index})"
        month = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, month_selector))
        )
        month.click()
        time.sleep(5)  # Esperar a que la página se actualice

        print(f"Extrayendo datos para el mes {month_index}")
        page_source = driver.page_source
        matches_df = extract_matches_from_html(page_source)
        all_matches_df = pd.concat([all_matches_df, matches_df], ignore_index=True)
        print(f"Datos extraídos para el mes {month_index}. Partidos en este mes: {len(matches_df)}")

        # Volver a abrir el calendario para la siguiente iteración
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "toggleCalendar"))).click()
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "datePicker")))
        
        month_index += 1
    
    except TimeoutException:
        print(f"No se encontró el mes {month_index}. Finalizando la extracción.")
        break
    except StaleElementReferenceException:
        print(f"Elemento obsoleto encontrado para el mes {month_index}. Reintentando...")
        continue
    except Exception as e:
        print(f"Error inesperado procesando el mes {month_index}: {str(e)}")
        print("Intentando continuar con el siguiente mes...")
        month_index += 1
        if month_index > 12:  # Asumiendo que no habrá más de 12 meses seleccionables
            break

Esperando que el calendario se abra...
Calendario abierto exitosamente.
Intentando procesar el mes 1
Extrayendo datos para el mes 1
Datos extraídos para el mes 1. Partidos en este mes: 36
Intentando procesar el mes 2
Extrayendo datos para el mes 2
Datos extraídos para el mes 2. Partidos en este mes: 36
Intentando procesar el mes 3
Extrayendo datos para el mes 3
Datos extraídos para el mes 3. Partidos en este mes: 18
Intentando procesar el mes 4
No se encontró el mes 4. Finalizando la extracción.


In [16]:

driver.quit()

In [ ]:

# Opcional: guardar en CSV
all_fixtures_df.to_csv("champions_league_fixtures_1.csv", index=False)
print("Datos guardados en 'champions_league_fixtures.csv'")

Datos guardados en 'champions_league_fixtures.csv'


In [80]:
print(len(all_fixtures_df)) 

90


In [81]:
print(f"Total de partidos extraídos: {len(all_fixtures_df)}")
all_fixtures_df.head()

Total de partidos extraídos: 90


,date,time_or_status,home_team,away_team,match_id,home_score,away_score,home_odds,draw_odds,away_odds
0,"Tuesday, Oct 01 2024",FT,Salzburg,Brest,1866220,0,4,N/A,N/A,N/A
1,"Tuesday, Oct 01 2024",FT,VfB Stuttgart,Sparta Prague,1866237,1,1,N/A,N/A,N/A
2,"Tuesday, Oct 01 2024",FT,PSV Eindhoven,Sporting CP,1866209,1,1,N/A,N/A,N/A
3,"Tuesday, Oct 01 2024",FT,Arsenal,Paris Saint-Germain,1866166,2,0,N/A,N/A,N/A
4,"Tuesday, Oct 01 2024",FT,Borussia Dortmund,Celtic,1866131,7,1,N/A,N/A,N/A


In [72]:
all_fixtures_df['date'].value_counts()

date
Tuesday, Oct 01 2024      9
Wednesday, Oct 02 2024    9
Tuesday, Oct 22 2024      9
Wednesday, Oct 23 2024    9
Tuesday, Nov 05 2024      9
Wednesday, Nov 06 2024    9
Tuesday, Nov 26 2024      9
Wednesday, Nov 27 2024    9
Tuesday, Sep 17 2024      6
Wednesday, Sep 18 2024    6
Thursday, Sep 19 2024     6
Name: count, dtype: int64

In [73]:
all_fixtures_df

,date,time_or_status,home_team,away_team,match_id,home_score,away_score,home_odds,draw_odds,away_odds
0,"Tuesday, Oct 01 2024",FT,Salzburg,Brest,1866220,0,4,N/A,N/A,N/A
1,"Tuesday, Oct 01 2024",FT,VfB Stuttgart,Sparta Prague,1866237,1,1,N/A,N/A,N/A
2,"Tuesday, Oct 01 2024",FT,PSV Eindhoven,Sporting CP,1866209,1,1,N/A,N/A,N/A
3,"Tuesday, Oct 01 2024",FT,Arsenal,Paris Saint-Germain,1866166,2,0,N/A,N/A,N/A
4,"Tuesday, Oct 01 2024",FT,Borussia Dortmund,Celtic,1866131,7,1,N/A,N/A,N/A
5,"Tuesday, Oct 01 2024",FT,Barcelona,BSC Young Boys,1866137,5,0,N/A,N/A,N/A
6,"Tuesday, Oct 01 2024",FT,Bayer Leverkusen,AC Milan,1866154,1,0,N/A,N/A,N/A
7,"Tuesday, Oct 01 2024",FT,Slovan Bratislava,Manchester City,1866104,0,4,N/A,N/A,N/A
8,"Tuesday, Oct 01 2024",FT,Inter,FK Crvena Zvezda,1866108,4,0,N/A,N/A,N/A
9,"Wednesday, Oct 02 2024",FT,Shakhtar Donetsk,Atalanta,1866172,0,3,N/A,N/A,N/A
